# About
This notebook is used to update `src/oge/reference_tables/manual/ba_reference.csv` using the FERC allowable BA entries list, which is updated on a quarterly basis.

Check https://www.ferc.gov/power-sales-and-markets/electric-quarterly-reports-eqr and look for the "Allowable Entries for Balancing Authorities and Hubs"

The most recent file uploaded is https://www.ferc.gov/sites/default/files/2020-05/allowable-entries1_0.xls

Also check the EIA-930 data reference tables and List of Balancing Authorities posted on https://www.eia.gov/electricity/gridmonitor/about

In [1]:
import os
import requests
import pandas as pd
import numpy as np

## Process Ferc data
If the "Allowable Entries for Balancing Authoritiesand Hubs" file is newer than 2020-05, download the file into the **"../../src/oge/reference_tables/** directory and perform the following pre-processing steps.

### Read the File

In [ ]:
ferc_bas = pd.read_excel(
    "../../src/oge/reference_tables/allowable-entries1_0.xls",
    parse_dates=True,
    skiprows=2,
    usecols="A,B,D,F,G",
    names=["ba_code", "ba_name", "us_ba", "activation_date", "retirement_date"],
)

### Remove HUB entries

In [3]:
ferc_bas = ferc_bas.query("ba_code != 'HUB'")

### Remove Duplicated Entry
The BA code BCHA is duplicated in the data frame. Keep first entry of 2 rows to be compatible with what is in the reference table.

In [ ]:
ferc_bas = ferc_bas.drop_duplicates(subset="ba_code", keep="first")
ferc_bas

,ba_code,ba_name,us_ba,activation_date,retirement_date
0,AEBN,"AESC, LLC - AEBN",US,2004-01-01,2006-07-01
1,AEC,"Alabama Electric Cooperative, Inc.",US,2004-01-01,NaT
2,AECI,"Associated Electric Cooperative, Inc.",US,2004-01-01,NaT
3,AEGL,"AESC, LLC - Gleason",US,2004-01-01,2006-07-01
4,AELC,"AESC, LLC - Lincoln Center",US,2004-01-01,2006-07-01
...,...,...,...,...,...
200,WPEK,Aquila Networks - Kansas,US,2004-01-01,2015-04-01
201,WPS,Wisconsin Public Service Corporation,US,2004-01-01,NaT
202,WR,Western Resources dba Westar Energy,US,2004-01-01,2015-04-01
203,WWA,"CECD, LLC",US,2013-07-01,NaT


## Read Reference Table

In [ ]:
# read existing file and drop ferc columns in it
ba_reference = pd.read_csv("../../src/oge/reference_tables/ba_reference.csv").drop(
    columns=["ba_name_ferc", "activation_date_ferc", "retirement_date_ferc"]
)
ba_reference

,ba_code,ba_name,ba_category,timezone_reporting_eia930,timezone_local,us_ba,activation_date,retirement_date,source,ba_number
0,AEBN,"AESC, LLC - AEBN",NaN,NaN,NaN,Yes,1/1/2004,7/1/2006,FERC,1
1,AEC,PowerSouth Energy Cooperative,NaN,US/Central,US/Central,Yes,1/1/2004,9/1/2021,EIA & FERC,2
2,AECI,"Associated Electric Cooperative, Inc.",NaN,US/Central,US/Central,Yes,1/1/2004,NaN,EIA & FERC,3
3,AEGL,"AESC, LLC - Gleason",NaN,NaN,NaN,Yes,1/1/2004,7/1/2006,FERC,4
4,AELC,"AESC, LLC - Lincoln Center",NaN,NaN,NaN,Yes,1/1/2004,7/1/2006,FERC,5
...,...,...,...,...,...,...,...,...,...,...
235,TXMS,No Balancing Authority - Texas,miscellaneous,US/Central,US/Central,Yes,NaN,NaN,NaN,948
236,UTMS,No Balancing Authority - Utah,miscellaneous,US/Mountain,US/Mountain,Yes,NaN,NaN,NaN,949
237,WAMS,No Balancing Authority - Washington,miscellaneous,US/Pacific,US/Pacific,Yes,NaN,NaN,NaN,953
238,WIMS,No Balancing Authority - Wisconsin,miscellaneous,US/Central,US/Central,Yes,NaN,NaN,NaN,955


## Update Reference Table

In [ ]:
# merge the ferc data into the manual table
ba_reference_updated = ba_reference.merge(
    ferc_bas,
    how="outer",
    on="ba_code",
    suffixes=(None, "_ferc"),
    validate="1:1",
)

# fill any missing data in the manual table with the data from ferc
for col in [
    "ba_name",
    "us_ba",
    "activation_date",
    "retirement_date",
]:
    ba_reference_updated[col] = ba_reference_updated[col].fillna(
        ba_reference_updated[f"{col}_ferc"]
    )

    # wherever the ferc entry does not match the EIA entry, keep the ferc entry too
    ba_reference_updated.loc[
        (ba_reference_updated[f"{col}_ferc"] == ba_reference_updated[col]),
        f"{col}_ferc",
    ] = np.NaN


ba_reference_updated["source"] = ba_reference_updated["source"].replace(
    {"left_only": "EIA", "right_only": "FERC", "both": "EIA & FERC"}
)
# drop the us_ba_ferc column
ba_reference_updated = ba_reference_updated.drop(columns=["us_ba_ferc"])

if ba_reference_updated["ba_code"].isna().sum() > 0:
    ba_reference_updated.loc[
        ba_reference_updated["ba_code"].isna() == True, "ba_code"
    ] = "NA"

ba_reference_updated

,ba_code,ba_name,ba_category,timezone_reporting_eia930,timezone_local,us_ba,activation_date,retirement_date,source,ba_number,ba_name_ferc,activation_date_ferc,retirement_date_ferc
0,AEBN,"AESC, LLC - AEBN",NaN,NaN,NaN,Yes,1/1/2004,7/1/2006,FERC,1,NaN,2004-01-01,2006-07-01
1,AEC,PowerSouth Energy Cooperative,NaN,US/Central,US/Central,Yes,1/1/2004,9/1/2021,EIA & FERC,2,"Alabama Electric Cooperative, Inc.",2004-01-01,NaT
2,AECI,"Associated Electric Cooperative, Inc.",NaN,US/Central,US/Central,Yes,1/1/2004,NaT,EIA & FERC,3,NaN,2004-01-01,NaT
3,AEGL,"AESC, LLC - Gleason",NaN,NaN,NaN,Yes,1/1/2004,7/1/2006,FERC,4,NaN,2004-01-01,2006-07-01
4,AELC,"AESC, LLC - Lincoln Center",NaN,NaN,NaN,Yes,1/1/2004,7/1/2006,FERC,5,NaN,2004-01-01,2006-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,WPS,Wisconsin Public Service Corporation,NaN,NaN,NaN,Yes,1/1/2004,NaT,FERC,207,NaN,2004-01-01,NaT
236,WR,Western Resources dba Westar Energy,NaN,NaN,NaN,Yes,1/1/2004,4/1/2015,FERC,208,NaN,2004-01-01,2015-04-01
237,WWA,"NaturEner Wind Watch, LLC",generation_only,US/Mountain,US/Mountain,Yes,7/1/2013,NaT,EIA & FERC,209,"CECD, LLC",2013-07-01,NaT
238,YAD,"Alcoa Power Generating, Inc. - Yadkin Division",generation_only,US/Eastern,US/Eastern,Yes,1/1/2004,NaT,EIA & FERC,210,"Yadkin, Inc.",2004-01-01,NaT


## Write File

In [ ]:
ba_reference_updated.to_csv(
    "../../src/oge/reference_tables/ba_reference_updated.csv", index=False
)